# 02 - About $\textsf{RLWE}$ and $\textsf{RGSW}$.
---

From <a href='https://eprint.iacr.org/2020/086'> Bootstrapping in FHEW-like Cryptosystems </a>

In [1]:
from note_include.elem.Ring import *
from note_include.elem.RLWE import *

---
#### Parameters

In [2]:
n = 16
q = 7681
std = 1

---
Simple Example of Ring

In [3]:
# Test for Ring
a = [i for i in range(n)]
b = [1 for _ in range(n)]
c = np.zeros(n)
c[1] = 1

ring_poly1    = Ring(n, q, a)
ring_poly2    = Ring(n, q, b)
ring_monomial = Ring(n, q, c)

print(ring_poly1 + ring_poly2)
print(ring_poly1 * ring_monomial)


R(n=16, q=7681, coeffs= 1 + 2x + 3x^2 + 4x^3 + 5x^4 + 6x^5 + 7x^6 + 8x^7 + 9x^8 + 10x^9 + 11x^10 + 12x^11 + 13x^12 + 14x^13 + 15x^14 + 16x^15)
R(n=16, q=7681, coeffs= 7666.0 + 0.0x + 1.0x^2 + 2.0x^3 + 3.0x^4 + 4.0x^5 + 5.0x^6 + 6.0x^7 + 7.0x^8 + 8.0x^9 + 9.0x^10 + 10.0x^11 + 11.0x^12 + 12.0x^13 + 13.0x^14 + 14.0x^15)


---
Simple example of RLWE

In [4]:
rlwe   = RLWE(n, q, std)
sk, pk = rlwe.keygen()

a                = [100 for _ in range(n)]
b                = [100 for _ in range(n)]

poly_a           = Ring(n, q, a)
poly_b           = Ring(n, q, b)

print('poly_a : ', poly_a)
print('poly_b : ', poly_b)
print('---------------------------------------------------------')

poly_a :  R(n=16, q=7681, coeffs= 100 + 100x + 100x^2 + 100x^3 + 100x^4 + 100x^5 + 100x^6 + 100x^7 + 100x^8 + 100x^9 + 100x^10 + 100x^11 + 100x^12 + 100x^13 + 100x^14 + 100x^15)
poly_b :  R(n=16, q=7681, coeffs= 100 + 100x + 100x^2 + 100x^3 + 100x^4 + 100x^5 + 100x^6 + 100x^7 + 100x^8 + 100x^9 + 100x^10 + 100x^11 + 100x^12 + 100x^13 + 100x^14 + 100x^15)
---------------------------------------------------------


----
Encryption and Decryption

In [5]:
ctxt_a           = rlwe.encrypt(poly_a, sk)
ctxt_b           = rlwe.encrypt(poly_b, sk)

ptxt_a           = rlwe.decrypt(ctxt_a, sk)
ptxt_b           = rlwe.decrypt(ctxt_b, sk)


print('ctxt_a : ', ctxt_a) # (a, b)
print('ctxt_b : ', ctxt_b) # (a, b)
print('---------------------------------------------------------')

print('ptxt_a : ',ptxt_a)
print('ptxt_b : ',ptxt_b)
print('---------------------------------------------------------')

ctxt_a :  (R(n=16, q=7681, coeffs= 1240 + 67x + 4310x^2 + 6733x^3 + 5010x^4 + 2294x^5 + 1154x^6 + 8x^7 + 4383x^8 + 5789x^9 + 4824x^10 + 6255x^11 + 1140x^12 + 5434x^13 + 4358x^14 + 6761x^15), R(n=16, q=7681, coeffs= 3869 + 4642x + 365x^2 + 7654x^3 + 581x^4 + 2375x^5 + 4991x^6 + 151x^7 + 5774x^8 + 5517x^9 + 7449x^10 + 3074x^11 + 3252x^12 + 1617x^13 + 2783x^14 + 1051x^15))
ctxt_b :  (R(n=16, q=7681, coeffs= 1904 + 5887x + 6845x^2 + 2042x^3 + 7332x^4 + 5489x^5 + 6938x^6 + 1083x^7 + 1179x^8 + 4427x^9 + 4469x^10 + 3484x^11 + 5973x^12 + 4455x^13 + 4359x^14 + 713x^15), R(n=16, q=7681, coeffs= 6024 + 2941x + 4589x^2 + 2037x^3 + 1601x^4 + 671x^5 + 6592x^6 + 10x^7 + 1017x^8 + 701x^9 + 5544x^10 + 6204x^11 + 5763x^12 + 3786x^13 + 4612x^14 + 133x^15))
---------------------------------------------------------
ptxt_a :  R(n=16, q=7681, coeffs= 98 + 100x + 103x^2 + 100x^3 + 101x^4 + 100x^5 + 100x^6 + 100x^7 + 101x^8 + 101x^9 + 100x^10 + 102x^11 + 99x^12 + 101x^13 + 100x^14 + 99x^15)
ptxt_b :  R(n=16, q

---
Homomorphic addition and constant addition

In [6]:
constant         = [10 for _ in range(n)]
poly_const       = Ring(n, q, constant)

ctxt_add         = rlwe.add_ctxt_ctxt(ctxt_a, ctxt_b)
ctxt_add_const   = rlwe.add_ctxt_ptxt(ctxt_b, poly_const)

ptxt_add         = rlwe.decrypt(ctxt_add, sk)
ptxt_add_const   = rlwe.decrypt(ctxt_add_const, sk)

print('ptxt_add       : ', ptxt_add)
print('ptxt_add_const : ', ptxt_add_const)
print('---------------------------------------------------------')

ptxt_add       :  R(n=16, q=7681, coeffs= 198 + 198x + 204x^2 + 200x^3 + 201x^4 + 201x^5 + 199x^6 + 200x^7 + 201x^8 + 199x^9 + 201x^10 + 200x^11 + 200x^12 + 202x^13 + 201x^14 + 198x^15)
ptxt_add_const :  R(n=16, q=7681, coeffs= 110 + 108x + 111x^2 + 110x^3 + 110x^4 + 111x^5 + 109x^6 + 110x^7 + 110x^8 + 108x^9 + 111x^10 + 108x^11 + 111x^12 + 111x^13 + 111x^14 + 109x^15)
---------------------------------------------------------


---
Homomorphic multiplication with plaintext polynomial with small magnitude coeffcients

Note that now we are in ring $\Z_q[x]/(x^n+1)$, so multiplication between two polynomials is negacyclic convolution.

In [7]:
a                = [10 for _ in range(n)]
small_const      = [2  for _ in range(n)]
poly_a           = Ring(n, q, a)
poly_small_const = Ring(n, q, small_const)

ctxt_a           = rlwe.encrypt(poly_a, sk)
ctxt_mul         = rlwe.mult_ring_ptxt(ctxt_a, poly_small_const)
ptxt_mul         = rlwe.decrypt(ctxt_mul, sk)

print("Homomorphic result : ", ptxt_mul)
print("Pure result        : ", poly_a * poly_small_const)

Homomorphic result :  R(n=16, q=7681, coeffs= 7413 + 7453x + 7497x^2 + 7533x^3 + 7577x^4 + 7617x^5 + 7645x^6 + 7677x^7 + 36x^8 + 76x^9 + 116x^10 + 156x^11 + 196x^12 + 232x^13 + 272x^14 + 308x^15)
Pure result        :  R(n=16, q=7681, coeffs= 7401.0 + 7441.0x + 7481.0x^2 + 7521.0x^3 + 7561.0x^4 + 7601.0x^5 + 7641.0x^6 + 0.0x^7 + 40.0x^8 + 80.0x^9 + 120.0x^10 + 160.0x^11 + 200.0x^12 + 240.0x^13 + 280.0x^14 + 320.0x^15)
